In [1]:
from heeps.config.read_config import read_config
from heeps.config.update_config import update_config
from heeps.util.img_processing import resize_cube
from astropy.io import fits
import numpy as np
import os

In [2]:
os.chdir(os.path.normpath(os.path.expandvars('$HOME/heeps_metis/input_files')))
npupil_L = update_config(**dict(read_config(), band='L'))['npupil']
npupil_N = update_config(**dict(read_config(), band='N2'))['npupil']
npupil = {'L':npupil_L, 'N2':npupil_N}
mask_L = fits.getdata('wavefront/cfull/mask_Cfull_20211007_%s_%s.fits'%('L', npupil_L)) > 0.5
mask_N = fits.getdata('wavefront/cfull/mask_Cfull_20211007_%s_%s.fits'%('N2', npupil_N)) > 0.5
mask = {'L':mask_L, 'N2':mask_N}
scao_L = fits.getdata('wavefront/cfull/cube_Cfull_20211007_3600s_300ms_0piston_meters_scao_only_%s_%s.fits'%('L', npupil_L))
scao_N = fits.getdata('wavefront/cfull/cube_Cfull_20211007_3600s_300ms_0piston_meters_scao_only_%s_%s.fits'%('N2', npupil_N))
scao = {'L':scao_L, 'N2':scao_N}
filename = 'wavefront/ncpa/%s/DIFF_rep_1_field_%s.fits.gz'
tag = {'L':'LM_20211122_2000pts', 'N2':'N_20211122_2000pts'}

In [3]:
for band in ['L', 'N2']:
    for i,field in enumerate(range(0, 2000)):
        if i % 100 == 0:
            print(i)
        ncpa = fits.getdata(filename%(tag[band], field))
        ncpa = resize_cube(ncpa, npupil[band])
        ncpa -= np.mean(ncpa[mask[band]]) # remove piston
        ncpa[~mask[band]] = 0             # mask with zeros
        stack = np.dstack([ncpa.T]*6).T
        if i == 0:
            cube = stack
        else:
            cube = np.vstack((cube, stack))
    cube += scao[band]
    fits.writeto('wavefront/cfull/cube_Cfull_20211007_3600s_300ms_0piston_meters_all_ncpa_%s_%s_rep_1_2000pts.fits'%
                 (band, npupil[band]), cube, overwrite=True)
    print('SCAO+NCPA: %.2f nm'%(np.mean([np.std(x[mask[band]]) for x in cube*1e9])))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
SCAO+NCPA: 165.76 nm
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
SCAO+NCPA: 127.54 nm
